In [ ]:
!pip install Kaggle
from google.colab import files
files.upload()

Saving kaggle_2.json to kaggle_2.json


{'kaggle_2.json': b'{"username":"pushkum","key":"8b0383a1515fe4ec355bc96ceb789cbb"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ruizgara/socofing

100% 838M/838M [00:05<00:00, 212MB/s]
100% 838M/838M [00:05<00:00, 158MB/s]


In [ ]:
!unzip /content/socofing.zip -d /content/socofing

Streaming output truncated to the last 5000 lines.
  inflating: /content/socofing/socofing/SOCOFing/Real/191__F_Left_index_finger.BMP  
  inflating: /content/socofing/socofing/SOCOFing/Real/191__F_Left_little_finger.BMP  
  inflating: /content/socofing/socofing/SOCOFing/Real/191__F_Left_middle_finger.BMP  
  inflating: /content/socofing/socofing/SOCOFing/Real/191__F_Left_ring_finger.BMP  
  inflating: /content/socofing/socofing/SOCOFing/Real/191__F_Left_thumb_finger.BMP  
  inflating: /content/socofing/socofing/SOCOFing/Real/191__F_Right_index_finger.BMP  
  inflating: /content/socofing/socofing/SOCOFing/Real/191__F_Right_little_finger.BMP  
  inflating: /content/socofing/socofing/SOCOFing/Real/191__F_Right_middle_finger.BMP  
  inflating: /content/socofing/socofing/SOCOFing/Real/191__F_Right_ring_finger.BMP  
  inflating: /content/socofing/socofing/SOCOFing/Real/191__F_Right_thumb_finger.BMP  
  inflating: /content/socofing/socofing/SOCOFing/Real/192__M_Left_index_finger.BMP  
  infla

In [ ]:
import os
import cv2
import random
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import layers, regularizers, optimizers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
img_size = 96

def load_data(path, train=True):
    print("Loading data from: ", path)
#     data = []
    IDs = []
    fingerNums = []
    images = []
    for img in os.listdir(path):
        imgname, ext = os.path.splitext(img)
        ID, etc = imgname.split('__')
        ID = int(ID) - 1 # to_categorical encodes starting from 0
        if train:
            _, lr, finger, _, _ = etc.split('_')
        else:
            _, lr, finger, _  = etc.split('_')
        if lr=='Left':
            base = 0 # left hand corresponding to 0-4
        else: base  = 5 # right hand corresponding to 5-9
        if finger=="little":
            fingerNum = base + 0
        elif finger=='ring':
            fingerNum = base + 1
        elif finger=='middle':
            fingerNum = base + 2
        elif finger=='index':
            fingerNum = base + 3
        else: fingerNum = base + 4
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        img_resize = cv2.resize(img_array, (img_size, img_size))
#         data.append([ID, fingerNum, img_resize])

        IDs.append(ID)
        fingerNums.append(fingerNum)
        images.append(img_resize)
    return IDs, fingerNums, images
#     return data

In [ ]:
Altered_path = "/content/socofing/SOCOFing/Altered/Altered-"
Real_path = "/content/socofing/SOCOFing/Real"

Easy_IDs, Easy_fingerNums, Easy_images = load_data(Altered_path+'Easy', train=True)
Medium_IDs, Medium_fingerNums, Medium_images = load_data(Altered_path+'Medium', train=True)
Hard_IDs, Hard_fingerNums, Hard_images = load_data(Altered_path+'Hard', train=True)
Real_IDS, Real_fingerNums, Real_images = load_data(Real_path, train=False)

Altered_IDs = np.concatenate([Easy_IDs, Medium_IDs, Hard_IDs], axis=0)
Altered_fingerNums = np.concatenate([Easy_fingerNums, Medium_fingerNums, Hard_fingerNums], axis=0)
Altered_images = np.concatenate([Easy_images, Medium_images, Hard_images], axis=0)

Loading data from:  /content/socofing/SOCOFing/Altered/Altered-Easy
Loading data from:  /content/socofing/SOCOFing/Altered/Altered-Medium
Loading data from:  /content/socofing/SOCOFing/Altered/Altered-Hard
Loading data from:  /content/socofing/SOCOFing/Real


In [ ]:
X_Altered, y_SubjectID_Altered, y_fingerNum_Altered = [], [], []

X_Altered = Altered_images.reshape(-1, img_size, img_size, 1) / 255.0

# Convert the labels to categorical
y_SubjectID_Altered = to_categorical(Altered_IDs, num_classes=600)
y_fingerNum_Altered = to_categorical(Altered_fingerNums, num_classes=10)

# Split the data into training and validation sets
X_SubjectID_train, X_SubjectID_val, y_SubjectID_train, y_SubjectID_val = train_test_split(
    X_Altered, y_SubjectID_Altered, test_size=0.2, random_state=2)
X_fingerNum_train, X_fingerNum_val, y_fingerNum_train, y_fingerNum_val = train_test_split(
    X_Altered, y_fingerNum_Altered, test_size=0.2, random_state=2)

In [ ]:
X_test, y_SubjectID_test, y_fingerNum_test = [], [], []

for SubjectID, fingerNum, feature in zip(Real_IDS, Real_fingerNums, Real_images):
    X_test.append(feature)
    y_SubjectID_test.append(SubjectID)
    y_fingerNum_test.append(fingerNum)

X_test = np.array(X_test).reshape(-1, img_size, img_size, 1)
X_test = X_test / 255.0

y_SubjectID_test = to_categorical(y_SubjectID_test, num_classes=600)
y_fingerNum_test = to_categorical(y_fingerNum_test, num_classes=10)

In [ ]:
print("Shapes:                  Feature shape    label shape")
print("----------------------------------------------------")
print("full SubjectID data:  ", X_Altered.shape, y_SubjectID_Altered.shape)
print("SubjectID_Train:      ", X_SubjectID_train.shape, y_SubjectID_train.shape)
print("SubjectID_Validation: ", X_SubjectID_val.shape, y_SubjectID_val.shape)
print("SubjectID_Test:       ", X_test.shape, y_SubjectID_test.shape)
print("----------------------------------------------------")
print("full fingerNum data:  ", X_Altered.shape, y_fingerNum_Altered.shape)
print("fingerNum_Train:      ", X_fingerNum_train.shape, y_fingerNum_train.shape)
print("fingerNum_Validation: ", X_fingerNum_val.shape, y_fingerNum_val.shape)
print("fingerNum_Test:       ", X_test.shape, y_fingerNum_test.shape)

# del Altered_data, Real_data, y_SubjectID_Altered # Free some memory again

Shapes:                  Feature shape    label shape
----------------------------------------------------
full SubjectID data:   (49270, 96, 96, 1) (49270, 600)
SubjectID_Train:       (39416, 96, 96, 1) (39416, 600)
SubjectID_Validation:  (9854, 96, 96, 1) (9854, 600)
SubjectID_Test:        (6000, 96, 96, 1) (6000, 600)
----------------------------------------------------
full fingerNum data:   (49270, 96, 96, 1) (49270, 10)
fingerNum_Train:       (39416, 96, 96, 1) (39416, 10)
fingerNum_Validation:  (9854, 96, 96, 1) (9854, 10)
fingerNum_Test:        (6000, 96, 96, 1) (6000, 10)


In [ ]:
nets = 2
model = [0] * nets
final_Dense_units = [600, 10]
model_name = ['SubjectID_Mod', 'FingerNum_Mod']
for i in range(nets):
    model[i] = Sequential(name=model_name[i])

    model[i].add(layers.Conv2D(32, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape = (96, 96, 1)))
    model[i].add(layers.BatchNormalization())
    model[i].add(layers.MaxPool2D((2, 2)))
    model[i].add(layers.Conv2D(64,(5, 5), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model[i].add(layers.BatchNormalization())
    model[i].add(layers.MaxPool2D((2, 2)))
    model[i].add(layers.Conv2D(128,(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model[i].add(layers.BatchNormalization())
    model[i].add(layers.MaxPool2D((2, 2)))
    model[i].add(layers.Dropout(0.3))
    model[i].add(layers.Flatten())
    model[i].add(layers.Dense(256, activation='relu'))
    model[i].add(layers.Dropout(0.4))
    model[i].add(layers.Dense(final_Dense_units[i], activation='softmax'))

    # Complete with Adam optimizer and entropy cost
    model[i].compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])
    model[i].summary()
#     model[i].save()

Model: "SubjectID_Mod"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 92, 92, 32)        832       
                                                                 
 batch_normalization_6 (Bat  (None, 92, 92, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 46, 46, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 42, 42, 64)        51264     
                                                                 
 batch_normalization_7 (Bat  (None, 42, 42, 64)        256       
 chNormalization)                                                
                                                     

In [ ]:
history = [0] * nets
CallBack = [0] * nets
ReduceLR_minlr = [1e-9, 1e-7]
epochs = 6
batch_size = 64
for i in range(nets):
    CallBack[i] = [
        callbacks.EarlyStopping(monitor='val_accuracy', patience=10, mode='max', verbose=1),
        callbacks.ModelCheckpoint('./model'+str(i)+'.keras', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True),
        callbacks.ReduceLROnPlateau(factor=0.1, patience=1, min_lr=ReduceLR_minlr[i], verbose=1),
        callbacks.TensorBoard(log_dir="./log_dir/"+model_name[i])]
history[0] = model[0].fit(X_SubjectID_train, y_SubjectID_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    validation_data = (X_SubjectID_val, y_SubjectID_val),
                    verbose = 1, callbacks= CallBack[0])

Epoch 1/6
616/616 [==============================] - ETA: 0s - loss: 6.4929 - accuracy: 0.0045
Epoch 1: val_accuracy improved from -inf to 0.00781, saving model to ./model0.keras
616/616 [==============================] - 300s 485ms/step - loss: 6.4929 - accuracy: 0.0045 - val_loss: 6.3210 - val_accuracy: 0.0078 - lr: 1.0000e-04
Epoch 2/6
616/616 [==============================] - ETA: 0s - loss: 5.9906 - accuracy: 0.0240
Epoch 2: val_accuracy improved from 0.00781 to 0.07469, saving model to ./model0.keras
616/616 [==============================] - 297s 482ms/step - loss: 5.9906 - accuracy: 0.0240 - val_loss: 5.5067 - val_accuracy: 0.0747 - lr: 1.0000e-04
Epoch 3/6
616/616 [==============================] - ETA: 0s - loss: 4.8719 - accuracy: 0.1166
Epoch 3: val_accuracy improved from 0.07469 to 0.35448, saving model to ./model0.keras
616/616 [==============================] - 313s 508ms/step - loss: 4.8719 - accuracy: 0.1166 - val_loss: 3.8223 - val_accuracy: 0.3545 - lr: 1.0000e-04
E

In [ ]:
# Delete the SubjectID dataset after it was used
del X_SubjectID_train, X_SubjectID_val, y_SubjectID_train, y_SubjectID_val
# Then reload fingerNum dataset before model fitting
X_fingerNum_train, X_fingerNum_val, y_fingerNum_train, y_fingerNum_val = train_test_split(
    X_Altered, y_fingerNum_Altered, test_size=0.2, random_state=2)

del X_Altered, y_fingerNum_Altered

history[1] = model[1].fit(X_fingerNum_train, y_fingerNum_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    validation_data = (X_fingerNum_val, y_fingerNum_val),
                    verbose = 1, callbacks= CallBack[1])

Epoch 1/6
616/616 [==============================] - ETA: 0s - loss: 1.7589 - accuracy: 0.4306
Epoch 1: val_accuracy improved from -inf to 0.59671, saving model to ./model1.keras
616/616 [==============================] - 311s 502ms/step - loss: 1.7589 - accuracy: 0.4306 - val_loss: 1.2973 - val_accuracy: 0.5967 - lr: 1.0000e-04
Epoch 2/6
616/616 [==============================] - ETA: 0s - loss: 1.2121 - accuracy: 0.6116
Epoch 2: val_accuracy improved from 0.59671 to 0.70509, saving model to ./model1.keras
616/616 [==============================] - 301s 488ms/step - loss: 1.2121 - accuracy: 0.6116 - val_loss: 0.9737 - val_accuracy: 0.7051 - lr: 1.0000e-04
Epoch 3/6
616/616 [==============================] - ETA: 0s - loss: 0.9038 - accuracy: 0.7245
Epoch 3: val_accuracy improved from 0.70509 to 0.82616, saving model to ./model1.keras
616/616 [==============================] - 314s 509ms/step - loss: 0.9038 - accuracy: 0.7245 - val_loss: 0.6697 - val_accuracy: 0.8262 - lr: 1.0000e-04
E

In [ ]:
model[0].save('SubjectID_Model.h5')
model[1].save('FingerNum_Model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model

model_SubjectID = load_model('SubjectID_Model.h5')
model_fingerNum = load_model('FingerNum_Model.h5')

In [ ]:
# Make prediction
img = cv2.imread('/content/socofing/socofing/SOCOFing/Altered/Altered-Easy/106__M_Right_index_finger_Obl.BMP', cv2.IMREAD_GRAYSCALE)
img_resize = cv2.resize(img, (img_size, img_size))
img_array = img_resize.reshape(1, img_size, img_size, 1) / 255.0
SubjectID_pred = model_SubjectID.predict(img_array)
fingerNum_pred = model_fingerNum.predict(img_array)


1/1 [==============================] - 0s 25ms/step


In [ ]:
np.argmax(SubjectID_pred)+1

106